# A simple decoding example

Written by Ladislas Nalborczyk (ladislas.nalborczyk@gmail.com)  
Last updated on February 1, 2024

In [15]:
! pip install git+https://github.com/lnalborczyk/meg_decoding_tools
! pip install git+https://github.com/mitchellostrow/DSA

  Cloning https://github.com/lnalborczyk/meg_decoding_tools to /private/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/pip-req-build-j4gpm6ii
  Running command git clone --filter=blob:none --quiet https://github.com/lnalborczyk/meg_decoding_tools /private/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/pip-req-build-j4gpm6ii
  Resolved https://github.com/lnalborczyk/meg_decoding_tools to commit 7a7d436a5d6375cefb20a873f3b165c977459a39
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/mitchellostrow/DSA to /private/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/pip-req-build-sw1ltvku
  Running command git clone --filter=blob:none --quiet https://github.com/mitchellostrow/DSA /private/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/pip-req-build-sw1ltvku
  Resolved https://github.com/mitchellostrow/DSA to commit 08551ebdf9d21cd5cba2af28e6ed4614fd9c2306
  Installing build de

In [16]:
import mne
import numpy as np
from meeg.decoding import time_decode
from meeg.decoding import prep_data_for_decoding

In [17]:
# importing the examplary data (MNE epochs of MEG data)
visual_4l2s = mne.read_epochs("../src/meeg/data/visual_4l2s.fif")
visual_7l2s = mne.read_epochs("../src/meeg/data/visual_7l2s.fif")

Reading /Users/ladislas/Desktop/meg_decoding_tools/examples/../src/meeg/data/visual_4l2s.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available


/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/ipykernel_46834/4242138223.py:2: RuntimeWarning: This filename (../src/meeg/data/visual_4l2s.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  visual_4l2s = mne.read_epochs("../src/meeg/data/visual_4l2s.fif")


Adding metadata with 13 columns
160 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/ladislas/Desktop/meg_decoding_tools/examples/../src/meeg/data/visual_7l2s.fif ...
    Found the data of interest:
        t =    -200.00 ...     800.00 ms
        0 CTF compensation matrices available


/var/folders/yp/p14xyqpj5l3bnhqhhfk5qhqr0000gn/T/ipykernel_46834/4242138223.py:3: RuntimeWarning: This filename (../src/meeg/data/visual_7l2s.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  visual_7l2s = mne.read_epochs("../src/meeg/data/visual_7l2s.fif")


Adding metadata with 13 columns
159 matching events found
No baseline correction applied
0 projection items activated


In [18]:
# for decoding, we'll keep only two categories and concatenate those
decoding_epochs = mne.concatenate_epochs(
    epochs_list=[visual_4l2s, visual_7l2s],
    add_offset=True, on_mismatch="raise", verbose=None
    )

# preparing MNE epochs and labels for decoding
X, y = prep_data_for_decoding(
    epochs=decoding_epochs,
    pca=True, n_components=60,
    moving_average=True, kernel_size=5,
    trials_averaging=False, ntrials=2, shuffling_or_not=True
)

Adding metadata with 13 columns
319 matching events found
No baseline correction applied
Original shape of the MEG data: (319, 306, 251)
Length of the labels to be predicted: 319


/Users/ladislas/Desktop/meg_decoding_tools/.venv/lib/python3.10/site-packages/meeg/decoding/prepare.py:170: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X = epochs.get_data()


Moving average applied with kernel size: 5
PCA explained variance:0.989


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.

  0%|          | Fitting SlidingEstimator : 0/251 [00:00<?,       ?it/s]
  4%|▍         | Fitting SlidingEstimator : 10/251 [00:00<00:02,   93.67it/s]

  0%|          | Fitting SlidingEstimator : 0/251 [00:00<?,       ?it/s]
  8%|▊         | Fitting SlidingEstimator : 21/251 [00:00<00:01,  141.38it/s]
 12%|█▏        | Fitting SlidingEstimator : 29/251 [00:00<00:01,  153.39it/s]

  1%|          | Fitting SlidingEstimator : 2/251 [00:00<00:07,   31.31it/s]
 16%|█▌        | Fitting SlidingEstimator : 39/251 [00:00<00:01,  170.77it/s]

  5%|▌         | Fitting SlidingEstimator : 13/251 [00:00<00:02,  111.52it/s]
 19%|█▉        | Fitting SlidingEstimator : 48/251 [00:00<00:01,  178.53it/s]

  9%|▉         | Fitting SlidingEstimator : 23/251 [00:00<00:01,  144.82it/s]


  0%|          | Fitting SlidingEstimator : 0/251 [00:00<?,       ?it/s]
 23%|██▎       | Fitting SlidingEstimator : 58/251 [00:00<00:01,  189.23it/

In [ ]:
# decoding time!
scores, time_decod = time_decode(meg_data=X, labels=y)

In [14]:
from meeg.plots import plotting_decoding_scores

# plotting the decoding accuracy over time
plotting_decoding_scores(
    decoding_scores=scores,
    x_ticks=decoding_epochs.times,
    end_stim=0.2,
    plot_title="Sensor space decoding"
)

ImportError: cannot import name 'compare_pca_through_time' from partially initialized module 'meeg.latent' (most likely due to a circular import) (/Users/ladislas/Desktop/meg_decoding_tools/.venv/lib/python3.10/site-packages/meeg/latent/__init__.py)